In [189]:
import pandas as pd
import numpy as np

import codecs, json

In [190]:
##################### SINGLE FILE #######################

In [191]:
with open('/Users/calmaleh/Desktop/school/project_course/jeppesen/ac_poor_4.bsad') as json_file:
    json_data = json.load(json_file)

frames = []
for j in range(len(json_data['tables'])):
    df = pd.DataFrame(np.array(json_data['tables'][j]['table'])[:,:], 
                           columns = json_data['tables'][j]['header']['variables'])
    df['MACH_MODE'] = json_data['tables'][j]['header']['mode']
    df['STATE'] = json_data['tables'][j]['header']['flightphase']
    if df['STATE'][0] == 'cruise':
        frames.append(df)

df = pd.concat(frames,ignore_index=True)
df = df[df['MACH_MODE'].str.contains("Mach")]
df['MACH_MODE'] = df['MACH_MODE'].map(lambda x: x.lstrip('Mach ')).astype(float)
df = df[['DISA','ALTITUDE','MASS','MACH_MODE','FUELFLOW']]
df

,DISA,ALTITUDE,MASS,MACH_MODE,FUELFLOW
420,0.0,0.0,21092.045205,0.75,0.176901
421,0.0,10000.0,21092.045205,0.75,0.162159
422,0.0,14000.0,21092.045205,0.75,0.167829
423,0.0,25000.0,21092.045205,0.75,0.183516
424,0.0,31000.0,21092.045205,0.75,0.150630
...,...,...,...,...,...
775,10.0,35000.0,10432.624510,0.80,0.125929
776,10.0,37000.0,10432.624510,0.80,0.117493
777,10.0,45000.0,10432.624510,0.80,0.090385
778,10.0,47000.0,10432.624510,0.80,0.086970


In [192]:
##################### ALL FILES #######################

In [193]:
## Print out every mode for each data file
for i in range(7):
    with open('/Users/calmaleh/Desktop/school/project_course/jeppesen/ac_poor_' + str(i+1) +'.bsad') as json_file:
        json_data = json.load(json_file)
        for i in range(8):
            print(json_data['tables'][i]['header']['mode'])
    print('\n')

300 KIAS/M0.80
1 Engine EROPS
2LRC OXY EROPS
Mach 0.80
Mach 0.83
Mach 0.86
Max Cruise Thrust
Max Range Cruise


250/300 KIAS/M0.78
1LRC EROPS OXY ICE
250 KIAS/M0.77
2LRC EROPS OXY ICE
300 KIAS/M0.80
300 KIAS/M0.82
Long Range Cruise
M0.78/300/250 KIAS


Normal Climb
1MCT EROPS ICE
2LRC EROPS OXY ICE
Long Range Cruise
Mach 0.75
Mach 0.80
Mach 0.84
Max Cruise Thrust


300 KIAS/M0.80
2MCT EROPS
3LRC OXY EROPS
Long Range Cruise
Mach 0.75
Mach 0.80
Max Cruise Thrust
M0.80/300 KIAS


250/300/M0.87
1 LRC EROPS
2 LRC OXY EROPS
Long Range Cruise
Mach 0.85
Mach 0.86
Mach 0.87
Mach 0.88


280 KIAS/M0.76
1ENG EROPS LRC
2ENG EROPS OXY LRC
Mach 0.76
Mach 0.78
Mach 0.80
Mach 0.82
300 KIAS/M0.82


250 KIAS/M0.70
1ENG EROPS M0.45
2ENG EROPS OXY
Long Range Cruise M0.76
Mach 0.78
Mach 0.80
Max Continuous Thrust Cruise
Max Range Cruise M0.74




In [194]:
## note: altitude 0 despite speed being high for some points?
df = pd.DataFrame([])
for i in range(7):
    with open('/Users/calmaleh/Desktop/school/project_course/jeppesen/ac_poor_' + str(i+1) +'.bsad') as json_file:
        json_data = json.load(json_file)
      
    frames = []
    for j in range(len(json_data['tables'])):
        temp_df = pd.DataFrame(np.array(json_data['tables'][j]['table'])[:,:], 
                               columns = json_data['tables'][j]['header']['variables'])
        temp_df['MACH_MODE'] = json_data['tables'][j]['header']['mode']
        temp_df['STATE'] = json_data['tables'][j]['header']['flightphase']
        if temp_df['STATE'][0] == 'cruise':
            frames.append(temp_df)

    temp_df = pd.concat(frames,ignore_index=True)
    
    temp_df = temp_df[temp_df['MACH_MODE'].str.contains("Mach")]
    temp_df['MACH_MODE'] = temp_df['MACH_MODE'].map(lambda x: x.lstrip('Mach ')).astype(float)
    
    temp_df = temp_df[['DISA','ALTITUDE','MASS','MACH_MODE','FUELFLOW']]
    
    df = pd.concat([df,temp_df])
    
df

,DISA,ALTITUDE,MASS,MACH_MODE,FUELFLOW
162,0.0,0.0,16193.247609,0.8,0.154221
163,0.0,10000.0,16193.247609,0.8,0.220622
164,0.0,19000.0,16193.247609,0.8,0.200967
165,0.0,27000.0,16193.247609,0.8,0.176775
166,0.0,33000.0,16193.247609,0.8,0.138850
...,...,...,...,...,...
1459,20.0,41000.0,4082.331330,0.8,0.046745
1460,20.0,43000.0,4082.331330,0.8,0.041138
1461,20.0,45000.0,4082.331330,0.8,0.036665
1462,20.0,47000.0,4082.331330,0.8,0.031625


In [195]:
def speed_converter(df):
    TROPOPAUSE_ALT = 11000.0
    STRATOSPHERE_ALT = 20000

    TROPOPAUSE_TEMP = 216.65
    STD_TEMP = 288.15

    STD_LAPSE_RATE = -0.0065

    SPEED_OF_SOUND = 340.294

    df['ISA_CONDITION'] = False
    df.loc[(0 <= df['ALTITUDE']) & (df['ALTITUDE'] <= TROPOPAUSE_ALT), 'ISA_CONDITION'] = True

    df['ISA'] = 0

    df.loc[df.ISA_CONDITION == False, 'ISA'] = TROPOPAUSE_TEMP
    df.loc[df.ISA_CONDITION == True, 'ISA'] = round(STD_TEMP + STD_LAPSE_RATE * df[df.ISA_CONDITION == True].ALTITUDE,2)
    df['SPEED_SOUND'] = SPEED_OF_SOUND * np.sqrt((df['ISA'] + df['DISA'])/ STD_TEMP)

    #df['MACH'] = round(df['MACH_'] / df['SPEED_SOUND'],4)
    df['TAS'] = round((df['MACH_MODE'] * df['SPEED_SOUND']), 1)
    
    return df.drop(['ISA_CONDITION', 'ISA','SPEED_SOUND', 'MACH_MODE'],axis=1)

In [196]:
df = speed_converter(df)
df

,DISA,ALTITUDE,MASS,FUELFLOW,TAS
162,0.0,0.0,16193.247609,0.154221,272.2
163,0.0,10000.0,16193.247609,0.220622,239.6
164,0.0,19000.0,16193.247609,0.200967,236.1
165,0.0,27000.0,16193.247609,0.176775,236.1
166,0.0,33000.0,16193.247609,0.138850,236.1
...,...,...,...,...,...
1459,20.0,41000.0,4082.331330,0.046745,246.7
1460,20.0,43000.0,4082.331330,0.041138,246.7
1461,20.0,45000.0,4082.331330,0.036665,246.7
1462,20.0,47000.0,4082.331330,0.031625,246.7
